Get the accuracy metrics associated to a  (218776, 59, 1) prediction

In [7]:
import pickle
import os
import numpy as np
import pandas as pd

# Assuming the script is located in the same directory as the 'results' folder
folder_path = os.path.join('CRN', 'results')
file_name = 'new_cancer_sim_2_2.p'

path = os.path.join(folder_path, file_name)

with open(path, 'rb') as f:
    results = pickle.load(f)

In [8]:
# read npy tile
tile = np.load('CRN\predictions_CRN.npy')
tile.shape

(218776, 59, 1)

In [10]:
results['test_data']['cancer_volume'].shape

(218776, 60)

In [21]:
results['test_data']['sequence_lengths']

array([ 1.,  1.,  1., ..., 59., 59., 59.])

In [25]:
# TODO: add counterfactuals benchmaring
# We just do the accuracy at each timestep and then we average the accuracy over all the timesteps

tests_num = len(results['test_data']['sequence_lengths']) // 4
assert len(results['test_data']['sequence_lengths']) % 4 == 0
right_answers = 0

y_predicted_list = []
y_true_list = []

for i in range(0, tests_num*4 , 4):
    assert results['test_data']['sequence_lengths'][i] == results['test_data']['sequence_lengths'][i+1]
    assert results['test_data']['sequence_lengths'][i] == results['test_data']['sequence_lengths'][i+2]
    assert results['test_data']['sequence_lengths'][i] == results['test_data']['sequence_lengths'][i+3]

    sequence_length = int(results['test_data']['sequence_lengths'][i])
    
    if sequence_length != 59:
        y_predicted = [tile[i][sequence_length], tile[i+1][sequence_length], tile[i+2][sequence_length], tile[i+3][sequence_length]]
        min_predicted_index = np.argmin(y_predicted)

        # get the index of the true min value
        y_true = [results['test_data']['cancer_volume'][i][sequence_length], results['test_data']['cancer_volume'][i+1][sequence_length], 
                results['test_data']['cancer_volume'][i+2][sequence_length], results['test_data']['cancer_volume'][i+3][sequence_length]]
        min_true_index = np.argmin(y_true)

        y_predicted_list.append(min_predicted_index)
        y_true_list.append(min_true_index)

        if min_predicted_index == min_true_index:
            right_answers += 1



y_predicted_list = np.array(y_predicted_list)
y_true_list = np.array(y_true_list)

from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
print('accuracy: ', accuracy_score(y_true_list, y_predicted_list))

# precision, recall, f1-score, support
from sklearn.metrics import classification_report
print(classification_report(y_true_list, y_predicted_list))



confusion_matrix(y_true_list, y_predicted_list)



accuracy:  0.8235676720373158
              precision    recall  f1-score   support

           0       0.67      0.82      0.74      5216
           1       0.00      0.00      0.00         0
           2       0.00      0.69      0.00        13
           3       0.98      0.82      0.89     48582

    accuracy                           0.82     53811
   macro avg       0.41      0.58      0.41     53811
weighted avg       0.95      0.82      0.88     53811



c:\Users\idormoy\anaconda3\envs\RA\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\idormoy\anaconda3\envs\RA\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\idormoy\anaconda3\envs\RA\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[ 4262,    10,     0,   944],
       [    0,     0,     0,     0],
       [    4,     0,     9,     0],
       [ 2079,    52,  6405, 40046]], dtype=int64)